<a href="https://colab.research.google.com/github/cristianbossolasco/scraper-agencias-viajes/blob/main/scraper_busplus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# script para que funcione selenium en colab

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.j7a3EJlwAD/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.iIZt4usjMF/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.IjoPplNLDV/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

# Scraper

## imports

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re

## funciones

In [3]:
url = 'https://raw.githubusercontent.com/cristianbossolasco/scraper-agencias-viajes/main/busplus_files/paradas.csv'
df_paradas = pd.read_csv(url, delimiter = ';', encoding = 'unicode_escape')
df_paradas.head()

,id,direccion,latitud,longitud,nombre
0,6,,-35137317600,-6.045238e+10,Bragado - Caminera (Buenos Aires - ARG)
1,7,,-35838385600,-6.187472e+10,Pehuajo - ESSO (Buenos Aires - ARG)
2,8,,-35636855700,-6.135356e+10,Carlos Casares - Shell (Buenos Aires - ARG)
3,10,,-34905308000,-5.795551e+10,La Plata - Terminal (Buenos Aires - ARG)
4,11,,-35451965600,-6.088127e+10,9 de Julio - Terminal (Buenos Aires - ARG)


In [15]:
def get_data(driver, origen_parada, nombre_origen_parada, destino_parada, nombre_destino_parada, fecha_ida):
  """
    Esta funcion es la encargada de hacer el scraper de un origen y destino puntual en una fecha dada 

    Args:
      origen_parada (str): id de la parade de origen
      nombre_origen_parada (str): El nombre de la parada de origen que sera el que figure en los registros del output
      destino_parada (str): id de la parada de destino
      nombre_destino_parada (str): El nombre de la parada de destino que sera el que figure en los registros del output
      fecha_ida (str): fecha de partida en el formato {year}/{month}/{day}

    Returns:
      lista de json con los viajes
  """

  url = f'https://checkout.busplus.com.ar/servicios?origen_parada={origen_parada}&destino_parada={destino_parada}&fecha_ida={fecha_ida}&fecha_vuelta=&pasajeros=1&empresa=&cupondescuento='
  driver.get(url)

  # get_attribute('innerHTML') para obtener el html

  clean_hora = lambda x: re.search(r'(\d+):(\d+)', str(x)).group()
  clean_precio = lambda x: re.search(r'(\d+)', str(x)).group()

  items = []
  filas  = driver.find_elements(By.CLASS_NAME,"fila_servicio.servicio_fila.idatr")
  # recorro los registros de viajes
  for fila in filas:
    if not "hide " in fila.get_attribute(name="class"):
      id = fila.get_attribute('id')

      empresa_img = fila.find_element(By.TAG_NAME,"td").find_element(By.TAG_NAME, 'img').get_attribute('src')
      salida = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[2]').text.replace('\n', ' ')
      llegada = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[4]').text.replace('\n', ' ')
      escala = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[5]/a').text.replace('\n', ' ')
      duracion = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[6]').text.replace('\n', ' ')
      duracion = clean_hora(duracion)
      categoria = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[7]/ul/li[1]').text
      butacas_libres = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[8]/ul/li[1]').text
      precio = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[9]/ul/li[1]/span').text
      precio = clean_precio(precio)

      item_json = {
          "origen_parada": origen_parada,
          "nombre_origen_parada": nombre_origen_parada,
          "destino_parada": destino_parada,
          "nombre_destino_parada": nombre_destino_parada,
          "fecha_ida": fecha_ida,
          "empresa_img": empresa_img,
          "salida": salida,
          "llegada": llegada,
          "escala": escala,
          "duracion": duracion,
          "categoria": categoria,
          "butacas_libres": butacas_libres,
          "precio": precio
      }

      items.append(item_json)

  return items

def get_instance_selenium():
  options = Options()
  options.add_argument("--headless")
  options.add_argument("--no-sandbox")
  options.headless = True
  driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)
  return driver


def scrapear_viajes(ls_origenes, ls_destinos, n_dias):
  """
    Busca todas las combinaciones entre los origenes y el destinos, y scrapena n_dias hacia adelante a partir de mañana

    Args:
      ls_origenes (list): lista con los id de origen
      ls_destinos (list): lista con los id de destino
      n_dias (list): cantidad de dias que se va a escrapear a partir de mañana

    Returns:
      lista de json con los viajes
  """

  getNombreparada = lambda id: df_paradas[df_paradas.id == id].nombre.values[0]
  driver = get_instance_selenium()
  resultado = []

  for iOrigen in range(len(ls_origenes)):

    id_origen = ls_origenes[iOrigen]
    nombre_origen = getNombreparada(id_origen)

    for iDestino in range(len(ls_destinos)):

      id_destino = ls_destinos[iDestino]
      if id_destino == id_origen: continue
      nombre_destino = getNombreparada(id_destino)
      print(nombre_destino)

      # incializo con la fecha de mañana
      date = datetime.now() + timedelta(1) 
      for j in range(n_dias): 

          year = str(date.year)
          month = ('00' + str(date.month))[-2:]
          day = ('00' + str(date.day))[-2:]
          fecha = f'{year}/{month}/{day}'
          
          result = get_data(driver, id_origen, nombre_origen, id_destino, nombre_destino, fecha)

          resultado.extend(result)
          date += timedelta(days=1)

  driver.quit()

  return resultado

## scrapeo

In [ ]:
#df_paradas.query('nombre.str.contains("(Entre Rios - ARG)")', engine='python')
ls_origen = [28] # Buenos aires
ls_destino = [176] # Parana
n_dias = 7
result = scrapear_viajes(ls_origen, ls_destino, n_dias)
df_results = pd.DataFrame.from_records(result)
df_results.to_excel('viajes.xlsx', encoding = 'utf-8-sig') 

In [ ]:
#df_paradas.query('nombre.str.contains("(Entre Rios - ARG)")', engine='python')
ls_origen = [28] # Buenos aires
ls_argentina = df_paradas.query('nombre.str.contains("ARG")', engine='python').id.tolist() # Toda argentina
ls_destino = ls_argentina
n_dias = 7
result = scrapear_viajes(ls_origen, ls_destino, n_dias)
df_results = pd.DataFrame.from_records(result)
df_results.to_excel('Argentina.xlsx', encoding = 'utf-8-sig')
